In [1]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data_related as data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init
import tester
import model_util as mu

In [2]:
first_np, second_np, label_np  = data.getData()

avg length / 5: 7


In [3]:
first_np, second_np = utils.process_splitted(first_np, second_np)

linearized complete!


In [4]:
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np, '/relation/ch2idx_relation')
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

done tokenizing both data!
saved ch2idx to file!
encoding comlete!


In [5]:
# pretrained_word2vec = pm.load_pretrained_model(ch2idx)
# pretrained_word2vec = torch.tensor(pretrained_word2vec)

In [6]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

In [7]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(
                                                                   train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=100000)

In [8]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/final/related')

In [9]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [10]:
# ====================
# set parameters here
# ====================

title = 'related_dc_1'
epochs = 40

input_size=len(ch2idx)
embed_dim = 300
hidden_size = 800
num_classes = 2
rnn_layers = 1

num_filters = [100, 200, 100]
kernel_sizes = [15, 21, 114]

dropout = 0.0

learning_rate = 0.001
weight_decay = 1e-4

model_name="RNN"
optim_name="Adam"
loss_fn_name="CEL"

pretrained_model=None
freeze_embedding=False,

In [11]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=input_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing without pretrained model!!!
OCR_rnn(
  (emb): Embedding(115, 300)
  (lstm1): RNN(
    (rnn): LSTM(300, 800, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(300, 800, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=3200, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [12]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.583478   | 67.691358  | 0.545200 | 71.49  | 28.35 
   2    |   0.517060   | 73.616049  | 0.511744 | 74.01  | 28.46 
   3    |   0.487426   | 75.919753  | 0.484632 | 76.27  | 28.53 
   4    |   0.468273   | 77.285185  | 0.476431 | 76.73  | 28.48 
   5    |   0.454305   | 78.274074  | 0.466840 | 77.74  | 28.26 
   6    |   0.442094   | 79.030864  | 0.462211 | 77.73  | 28.70 
   7    |   0.431807   | 79.807407  | 0.451096 | 79.04  | 28.78 
   8    |   0.421830   | 80.323457  | 0.455139 | 78.31  | 28.84 
   9    |   0.412423   | 80.859259  | 0.443124 | 78.91  | 28.99 
  10    |   0.404210   | 81.345679  | 0.436390 | 79.62  | 28.98 
  11    |   0.396488   | 81.902469  | 0.446279 | 78.72  | 28.94 
  12    |   0.388305   | 82.339506  | 0.447480 | 78.52  | 29.28 
  13    |   0.382847   | 82.681481  | 0.435802 | 79.4

In [13]:
mu.saveModel(title, model)

In [14]:
model = mu.getModel(title)
print(model)

OCR_rnn(
  (emb): Embedding(115, 300)
  (lstm1): RNN(
    (rnn): LSTM(300, 800, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(300, 800, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=3200, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.0, inplace=False)
  (dp2): Dropout(p=0.0, inplace=False)
)


In [15]:
loss, acc = tester.test(test_dataloader=test_dataloader,
                        device=device,
                        model=model,
                        title=title)

test loss:  0.5759917970746755
test acc:  78.75
saved precision and recall results to file!


In [16]:
with open('../result/final_related', 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [17]:
mu.graphModel(train_dataloader, model, writer)

uploaded model graph to tensorboard!


In [18]:
max_len

21